## Homework 2

#### 1. Import libraries and read data

In [40]:
import pandas as pd
import altair as alt
import numpy as np
from datetime import datetime
from statistics import mean

In [41]:
df = pd.read_excel('2014-2020.xlsx')

#split the time and date column into separate columns
df[['Day', 'Month', 'Year']] = df['Час/Дата'].str.split('.', 2, expand=True)
df[['Hour', 'Day']] = df['Day'].str.split('-', 1, expand=True)



#add column - day of the year
df['DayOfYear'] = df['Час/Дата'].str.split('-', 2, expand=True)[1]
for i in range(len(df['DayOfYear'])):
    df['DayOfYear'][i] = pd.to_datetime(df['DayOfYear'][i], dayfirst=True).timetuple().tm_yday
df


,Час/Дата,AES,TEC,VDE,TES,GES,GAES_GEN,CONSUMPTION,GAES_PUMP,UK_BLR_RUS,UK_EURO,UK_MLD,Unnamed: 12,Day,Month,Year,Hour,DayOfYear
0,24-31.12.2020,9235,2039,621,4942,385.0,0,16693,-405.0,-84.0,-11.0,-29.0,NaN,31,12,2020,24,366
1,23-31.12.2020,9221,2159,707,5549,470.0,0,17805,0.0,-43.0,-212.0,-46.0,NaN,31,12,2020,23,366
2,22-31.12.2020,9249,2377,709,5906,1000.0,0,18870,0.0,-13.0,-328.0,-30.0,NaN,31,12,2020,22,366
3,21-31.12.2020,9256,2499,702,6329,909.0,322,19887,0.0,0.0,-64.0,-66.0,NaN,31,12,2020,21,366
4,20-31.12.2020,9213,2521,702,6640,823.0,602,20387,0.0,-29.0,-48.0,-37.0,NaN,31,12,2020,20,366
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61363,5-01.01.2014,10427,2826,49,5506,274.0,0,18072,-488.0,-90.0,-421.0,-11.0,NaN,01,01,2014,5,1
61364,4-01.01.2014,10475,2827,42,5524,185.0,0,18453,0.0,-175.0,-399.0,-26.0,NaN,01,01,2014,4,1
61365,3-01.01.2014,10515,2837,54,5865,355.0,0,19107,0.0,-59.0,-400.0,-60.0,NaN,01,01,2014,3,1
61366,2-01.01.2014,10606,2822,49,6038,863.0,0,19665,0.0,-200.0,-409.0,-104.0,NaN,01,01,2014,2,1


#### 2. Як змінювалась структура електрогенерації роками?
Тут нам важливо порівняти різницю структури по роках, щоб побачити як вона зміювалася. Оскільки на лекціях згадувалося, що найчастіше ми порівнюємо за допомогою barcharts, то цей тип візуалізації першим спав на думку при аналізі цього питання. Тут як категорію для кожного стовпчика ми можемо взяти рік, а в довжину стовпчика закласти суму генерації електроенергії з кожної станції за рік. І також додатково в кожному стовпчику розподілити по кольорах яка електростанція скільки згенерувала енергії (АЕС, ТЕЦ, ВДЕ, ТЕС, ГЕС, ГАЕС), щоб чітко побачити саме структуру кожного року. Тобто в результаті вийде Stacked barchart. 
В такому випадку проблемно може бути визначити скільки саме згенерувала кожна електростанція в межах одного року, адже кольорами та розмірами частин кожного стовпчика ми зможемо відобразити лише загальну картину. Тому як рішення тут можна додати toolip до кожного стовпчика, щоб можна було глянути на чіткі числа або ж додати підписи на самому барчарті. Я вирішила додати toolip, оскільки кожен стовпчик ділиться на досить багато частин (у нас є аж 6 складових генерації) і підписування кожної з них на самому чарті засмітить його і зробить його важким для сприйняття. А toolip навпаки додасть інтерактивності та допоможе надати чітку інформацію.
Як альтернативу я розглядала створення окремих фасетів для кожного року, в такому варіанті краще видно розподл структури окремов кожному році, але складніше порівняти роки між собою, тому вирішила робити stacked barchart. 

In [42]:
#create a new pandas dataframe as a basis for our barchart
columns = ['Year', 'Station', 'MW']
df1 = pd.DataFrame(columns = columns)
rows = []

for y in df['Year'].unique():
    for s in  ['AES', 'TEC', 'VDE', 'TES', 'GES', 'GAES_GEN']:
        sum_MW = df.loc[df['Year'] == str(y), s].sum()
        r = [y, s, sum_MW]
        z = zip(columns, r)
        row_dict = dict(z)
        rows.append(row_dict)
        
df1 = df1.append(rows)

#build a barchart
chart1 = alt.Chart(df1).mark_bar().encode(
    x = alt.X('Year:O'), y = alt.Y('MW:Q'),
    tooltip = alt.Tooltip('MW:Q'),
    color = alt.Color('Station:N'),
).properties(width = 650, height = 500, title = "Annual generation of power by station, in MW")
chart1

alt.Chart(...)

#### 3. Як залежить споживання електроенергії від дня року та години доби?
На мою думку, тут нас не цікавить конкретно кожна година та кожен день року, оскільки цих значень дуже багато. Тут зручніше загально візуалізувати взаємозвязок за допомогою розподілу графіка на прямокутники та відображати кольором наскільки більше чи менше споживання електроенергії відбувається в певний період дня і року. Першим моїм варіантом було відобразити це на heatmap, але на жаль мені не вдалося цього зробити, тому спробувала відобразити залежність за допомогою scatterplot, на якому можна навести мишку на точку і побачити в який день року і в яку годину який саме середній consumption.

In [43]:
df2 = df.drop(columns = ['AES', 'TEC', 'VDE', 'TES', 'GES', 'GAES_GEN', 'GAES_PUMP', 'UK_BLR_RUS', 'UK_EURO', 'UK_MLD', 'Unnamed: 12', 'Day', 'Month', 'Year'])
df2 = df2.groupby(['DayOfYear', 'Hour'])['CONSUMPTION'].apply(list).reset_index()
df2['CONSUMPTION_AVG'] = [round(mean(el),2) for el in df2['CONSUMPTION']]
df2

,DayOfYear,Hour,CONSUMPTION,CONSUMPTION_AVG
0,1,1,"[16150, 17478, 16546, 17760, 17686, 21211, 20586]",18202.43
1,1,10,"[15402, 16875, 15081, 16609, 16817, 20177, 19014]",17139.29
2,1,11,"[15735, 17471, 15719, 17151, 17259, 20658, 19898]",17698.71
3,1,12,"[16052, 17591, 16048, 17380, 17318, 20496, 20325]",17887.14
4,1,13,"[16070, 17553, 15874, 17257, 17436, 20523, 20406]",17874.14
...,...,...,...,...
8779,366,5,"[15446, 16160]",15803.00
8780,366,6,"[16347, 16511]",16429.00
8781,366,7,"[17000, 16959]",16979.50
8782,366,8,"[17726, 17500]",17613.00


In [44]:
alt.Chart(df2.sample(5000)).mark_circle(size=60).encode(
    x='DayOfYear',
    y='Hour',
    color='CONSUMPTION_AVG:Q',
    tooltip=['DayOfYear','Hour','CONSUMPTION_AVG:Q']).properties(width=550,height=380,title="Consumption dependency from day of year and hour of day")

alt.Chart(...)

#### 4. Як змінюється генерація електроенергії з різних джерел впродовж доби?
Тут знову ж таки важливий аспект - порівняти генерацію електроенергії залежно від часу дня. Моїм рішенням є відобразити це на stacked barchart. На ньому буде легко побачити яка кількість електроенергії в середньому генерується з якої станції і в якій годині.

In [45]:
columns = ['Hour', 'Station', 'Average Generation']

df2 = pd.DataFrame(columns = columns)
rows2 = []

for h in range(24, 0, -1):
    for s in  ['AES', 'TEC', 'VDE', 'TES', 'GES', 'GAES_GEN']:
        m = df.loc[df['Hour'] == str(h), s].mean()
        r = [h, s, m]
        z = zip(columns, r)
        r_dict = dict(z)
        rows2.append(r_dict)

df2 = df2.append(rows2)

alt.Chart(df2).mark_bar().encode(x = alt.X('Hour:O'),
                                 y = alt.Y('Average Generation:Q'),
                                 tooltip = alt.Tooltip('Average Generation'),
                                 color = alt.Color('Station:N'),).properties(width = 950, height = 600, title = "How average generation changes throughout the day by station?")

alt.Chart(...)

#### 5. Як змінюється споживання електроенергії впродовж доби у розрізі місяців року та пір року?

#### 6. Як змінюється споживання електроенергії впродовж тижня?